# 5. Pandas - GroupBy

[Group By: split-apply-combine](http://pandas.pydata.org/pandas-docs/stable/groupby.html)

In [ ]:
%pylab inline
from pandas import Series, DataFrame
import pandas as pd

## GroupBy技術
分組運算是一種 Split-Apply-Combine的過程，類似於MapReduce的模式
<div style="width:400px;height:400px;float:left">
![Pandas GroupBy](http://i.stack.imgur.com/sgCn1.jpg)
</div>

In [ ]:
df = DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                'key2' : ['one', 'two', 'one', 'two', 'one'],
                'data1' : np.random.randn(5),
                'data2' : np.random.randn(5)}, columns = ['key1', 'key2', 'data1', 'data2'])
df

In [ ]:
# 使用 groupby方法
grouped = df.data1.groupby(df.key1)
grouped
# 產生一個 SeriesGroupBy物件

In [ ]:
grouped.size()

In [ ]:
# 用GroupBy物件的 mean()方法
# mean()方法是一種 聚合運算
grouped.mean()

#### 分組所依據的鍵，可以是任何長度的數組，且可以有多層

In [ ]:
# 也可以建立多層次的分組
grouped = df.data1.groupby([df.key1, df.key2])
grouped.size()

In [ ]:
grouped.mean()

In [ ]:
grouped.mean().unstack('key1')

In [ ]:
# 也可以對多個 columns同時做分組統計運算
df.groupby(df.key1).mean()

In [ ]:
# 也可以直接以 column索引的名稱來指定分組
df.groupby(['key1', 'key2']).mean()

In [ ]:
# GroupBy 的 size()方法，傳回各分組的大小
df.groupby(['key1', 'key2']).size()

### 對分組進行迭代

In [ ]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)
# 所以分組的結果，是拆分為多個 DataFrame    

In [ ]:
# 依照多重鍵分組，groupby元素元組的第一個元素是 多重鍵的 元組
for name, group in df.groupby(['key1', 'key2']):
    print(name)
    print(group)

### 選取一個或一組columns

In [ ]:
df

In [ ]:
df.groupby('key1')['data1']
# 等同於
df['data1'].groupby(df['key1'])

df.groupby('key1')['data2']
# 等同於
df[['data2']].groupby(df['key1'])

In [ ]:
# 有時候只需要對部分的資料列進行聚合
df.groupby(['key1', 'key2'])[['data2']].mean()
# 傳回 DataFrame

In [ ]:
df.groupby(['key1', 'key2'])['data2'].mean()
# 傳回 Series

### 通過字典或Series進行分組

In [ ]:
people = DataFrame(np.random.randn(5, 5),
                   columns=['a', 'b', 'c', 'd', 'e'],
                   index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people.ix[2:3, ['b', 'c']] = np.nan
people

In [ ]:
# 已經知道 列的分組關係
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
           'd': 'blue', 'e': 'red', 'f' : 'orange'}
#只需要將mapping關係的字典傳給 groupby()
grouped_by_column = people.groupby(mapping, axis = 1)
grouped_by_column.sum()

In [ ]:
map_series = Series(mapping)
map_series

In [ ]:
# 也可以將mapping關係的Series物件傳給 groupby()
grouped_by_column = people.groupby(map_series, axis = 1)
grouped_by_column.sum()

### 透過函數進行分組

In [ ]:
people

In [ ]:
# 被當作分組鍵的函數都會在各個索引值上被調用一次，返回值就被當作分組名稱
people.groupby(len).mean()

In [ ]:
# 函數、列表、字典、Series都可以混用，因為最後都會被轉換為數組
key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).min()

In [ ]:
# 根據索引級別分組
# 要依據層次化索引來分組聚合，只需要透過 level參數即可
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                     [1, 3, 5, 1, 3]], names=['cty', 'tenor'])

hier_df = DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

In [ ]:
hier_df.groupby(level = 'cty', axis = 1).count()

## <font color='red'>數據聚合 (pandas.core.groupby.DataFrameGroupBy.aggregate() )</font>

In [ ]:
# 可以自訂一聚合方法。聚合方法會對每一個分組之後的group操作一次
df

In [ ]:
grouped = df.groupby('key1')
for name, group in grouped:
    print(name)
    print(group)

In [ ]:
type(grouped)

In [ ]:
for name, group in grouped['data1']:
    print(name)
    print(group)

In [ ]:
# Series, DataFrame的方法都可以施加在 group上
# quantile 是 Series的方法
grouped = df.groupby('key1')
grouped['data1'].quantile(0.9)

In [ ]:
# 透過 aggregate()方法，可以使用自訂函式
def peak_to_peak(arr):
    return arr.max() - arr.min()

grouped.aggregate(peak_to_peak)

# 會對每一個 pandas.core.groupby.DataFrameGroupBy(grouped)中的 DataFrame 中的 Series 做一次指定的 aggregate (在這邊是 peak_to_peak()) 運算

In [ ]:
# describe 也可以用
grouped.describe()

In [ ]:
grouped.mean()
# 會對每一個 pandas.core.groupby.DataFrameGroupBy中的 DataFrame 中的 Series 做一次指定的 aggregate (在這邊是 mean()) 運算

### 面向列的多函數應用

In [ ]:
tips = pd.read_csv('../data/tips.csv')
tips['tip_total_ratio'] = tips['tip']  / tips['total_bill'] 
tips[:5]

In [ ]:
# 對不同的列使用不同的聚合函數
grouped = tips.groupby(['sex', 'smoker'])
grouped_pct = grouped['tip_total_ratio']
for name, group in grouped_pct:
    print(name)
    print(group.tail(3))

[What is the difference between pandas agg and apply function?](http://stackoverflow.com/questions/21828398/what-is-the-difference-between-pandas-agg-and-apply-function)

In [ ]:
grouped_pct.agg('mean')

In [ ]:
grouped_pct.aggregate('mean')

In [ ]:
# 傳入一組函數或函數名，得到的DataFrame的列就會以相應的函數命名
grouped_pct.agg(['mean', 'std', peak_to_peak])

In [ ]:
# 如果傳入一個由(name, function)的元組列表，則各元組的第一個元素就會被當作DataFrame的 column名稱
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

In [ ]:
# 對於 DataFrame，還可以定義使用多個函數
functions = ['count', 'mean', 'max']
result = grouped['tip_total_ratio', 'total_bill'].agg(functions)
result

In [ ]:
result['tip_total_ratio']

In [ ]:
# 自訂一結果的列名稱
functions = [('Counts', 'count'), ('Mean', 'mean'), ('Max', 'max')]
result = grouped['tip_total_ratio', 'total_bill'].agg(functions)
result

In [ ]:
# 對於 DataFrame，還可以定義不同列使用不同的函數
# 傳入一個名稱與函數的字典
functions = {'tip_total_ratio':  np.max, 'total_bill': np.min}
result = grouped.agg(functions)
result

In [ ]:
# 對於 DataFrame，還可以定義不同列使用不同的函數
functions = {'tip_total_ratio': (np.max,  np.min), 
             'size': ['sum', 'min']}
result = grouped.agg(functions)
result

### 以 無索引 的形式返回聚合數據

In [ ]:
# 透過 as_index = False，分組鍵不要成為索引
tips.groupby(['sex', 'smoker'], as_index = False).mean()

In [ ]:
tips.groupby(['sex', 'smoker']).mean()

## 分組級運算和轉換

In [ ]:
# 聚合運算 是數據轉換的一種特例
# 為df增加一列 用於存放各索引分組平均值
df

In [ ]:
# 計算分組mean
k1_means = df.groupby('key1').mean().add_prefix('mean_')
k1_means

In [ ]:
# merge
pd.merge(df, k1_means, left_on = 'key1', right_index = True)

In [ ]:
# 使用 transform()
people

In [ ]:
key = ['one', 'two', 'one', 'two', 'one']
people.groupby(key).mean()

In [ ]:
# 使用 transform()，將分組結果又放到各個row中(使用廣播的方式)
people.groupby(key).transform(np.mean)

In [ ]:
# 可以套用各種自訂函式
# 距平均化函數
def demean(arr):
    return arr - arr.mean()

demeaned = people.groupby(key).transform(demean)
demeaned

In [ ]:
demeaned.groupby(key).transform(np.mean).applymap(lambda x: '{0:.5f}'.format(x))

### <font color='red'>apply: 一般性的 '拆分-應用-合併'</font>

[Difference between map, applymap and apply methods in Pandas](http://stackoverflow.com/questions/19798153/difference-between-map-applymap-and-apply-methods-in-pandas)  
apply: 對 整個DataFrame(單一group) 實施一次  
applymap: 對 DataFrame 的每個 儲存格 實施一次  
map: 是 Series 的 function，對 Series 的每個 數值 實施一次  

In [ ]:
# apply 會將資料拆分成多個片段，對各個片段調用函式，最後再組合各個結果
def top(df, n = 5, column = 'tip_total_ratio'):
    return df.sort_values(by = column)[-n:]

In [ ]:
tips.tail()

In [ ]:
top(tips, n = 6)

In [ ]:
# 使用 apply() 來施加 自訂函式
tips.groupby('smoker').apply(top)

In [ ]:
# 自訂函式所需要的參數，可以放在後面一起傳入
tips.groupby(['smoker', 'day']).apply(top, n = 1, column = 'total_bill')

In [ ]:
result = tips.groupby(['smoker',])['total_bill'].describe()
result

In [ ]:
result.unstack('smoker')

#### 禁止分組鍵 (group_keys = False)

In [ ]:
# 設定 group_keys = False，不讓分組鍵成為row索引
tips.groupby('smoker', group_keys = False).apply(top)

In [ ]:
tips.groupby('smoker', group_keys = True).apply(top)

### 範例: 分組加權平均數和相關係數

In [ ]:
df = DataFrame({'category': ['a', 'a', 'a', 'a', 'b', 'b', 'b', 'b'],
                'data': np.random.randn(8),
                'weights': np.random.rand(8)})
df

In [ ]:
# 計算分組加權平均數
get_wavg = lambda g: np.average(g.data * g.weights)

# 每個分組施以 get_wavg
df.groupby('category').apply(get_wavg)

In [ ]:
# Yahoo Finance 
close_px = pd.read_csv('../data/stock_px.csv',
                       parse_dates = True, index_col = 0)
close_px[:6]

In [ ]:
# 計算 日收益率 與 SPX之間的年度相關係數組成的DataFrame
rets = close_px.pct_change().dropna()
rets[:6]

In [ ]:
# 與 SPX之間的相關係數
spx_corr = lambda g: g.corrwith(g.SPX)

# 以年度區分
by_year = rets.groupby(lambda x: x.year)

# 計算分組與 SPX的 corr
by_year.apply(spx_corr)

In [ ]:
# 也可以計算 列與列之間的相關係數
by_year.apply(lambda g: g.AAPL.corr(g.MSFT))

## <font color='red'>透視表(pivot table)和交叉表(cross-tabulation, 或稱 crosstab)</font>

In [ ]:
tips = pd.read_csv('../data/tips.csv')
tips['tip_pct'] = tips['tip']  / tips['total_bill'] 
tips[:5]

In [ ]:
# DataFrame 本身就有 pivot_table()方法，預設的 aggregate function 是 average
tips.pivot_table(index = ['sex', 'smoker'])

In [ ]:
# 只聚合 tip_pct, size，而且想根據day來分組
# margins = True , 添加分項小計
tips.pivot_table(values = ['tip_pct', 'size'], index = ['sex', 'day'], columns = 'smoker', margins = True) 

In [ ]:
tips.pivot_table(values = ['tip_pct'], index = ['sex', 'smoker'], columns = 'day', margins = True) 

In [ ]:
# 也可傳入指定的 aggregate function (參數 aggfunc)
tips.pivot_table(values = ['tip_pct'], index = ['sex', 'smoker'], columns = 'day', margins = True, aggfunc = len) 

In [ ]:
# 如果存在空的組合(NA)，可以指定 fill_value參數，自動填入空缺值
tips.pivot_table(values = ['size'], index = ['time', 'sex', 'smoker'], columns = 'day', margins = True, aggfunc = sum, fill_value = 0) 

### 交叉表(crosstab)
用於計算 分組頻率 的特殊 透視表(pivot)

In [ ]:
data = DataFrame(
                {'Sample': list(range(1, 11)),
                 'Gender': [random.choice(['Female', 'Male']) for i in range(10)],
                 'Handedness': [random.choice(['Right-handed', 'Left-handed']) for i in range(10)]
                }, 
                columns = ['Sample', 'Gender', 'Handedness'])
data

In [ ]:
# 用 crosstab() 方法
pd.crosstab(data.Gender, data.Handedness, margins = True)

In [ ]:
# crosstab()方法的參數值可以是 數組
pd.crosstab(index = [tips.time, tips.day], columns = tips.smoker, margins = True)